In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load Libs

In [2]:
!pip install mediapipe
!pip install visualkeras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.7/993.7 kB 24.5 MB/s eta 0:00:00


In [3]:
import os
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import visualkeras
from sklearn.preprocessing import LabelEncoder
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from sklearn.preprocessing import StandardScaler

# Load Dataset

In [4]:
IMAGE_DATASET_PATH = '/content/drive/MyDrive/NULP/Masters/Thesis/CodeBase/asl_dataset'
KEYPOINTS_DATASET_PATH = '/content/drive/MyDrive/NULP/Masters/Thesis/CodeBase/asl_dataset_with_landmarks.csv'

In [5]:
# Ініціалізація MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

def load_keypoints_dataset():
  # Список класів (букв ASL)
  classes = sorted(os.listdir(IMAGE_DATASET_PATH))

  # Ініціалізація списків для збереження даних та міток
  data = []
  labels = []

  # Функція для обробки одного зображення
  def process_image(image_path):
      image = cv2.imread(image_path)
      if image is None:
          return None
      image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      result = hands.process(image_rgb)
      if result.multi_hand_landmarks:
          hand_landmarks = result.multi_hand_landmarks[0]
          landmarks = []
          for lm in hand_landmarks.landmark:
              landmarks.extend([lm.x, lm.y, lm.z])
          return landmarks
      else:
          return None

  # Ітерація по всіх класах та зображеннях
  for label in classes:
      class_path = os.path.join(IMAGE_DATASET_PATH, label)
      if not os.path.isdir(class_path):
          continue
      print(f'Processing class: {label}')
      for img_name in os.listdir(class_path):
          img_path = os.path.join(class_path, img_name)
          landmarks = process_image(img_path)
          if landmarks:
              data.append(landmarks)
              labels.append(label)

  # Закриття MediaPipe
  hands.close()

  # Перетворення списків у DataFrame
  data_df = pd.DataFrame(data)
  data_df['label'] = labels

  return data_df

In [7]:
if os.path.exists(KEYPOINTS_DATASET_PATH):
  df = pd.read_csv(KEYPOINTS_DATASET_PATH)

else:
  df = load_keypoints_dataset()
  df.to_csv(KEYPOINTS_DATASET_PATH, index=False)

print(f'Data shape: {df.shape}')


Data shape: (1621, 64)


## Keypoints visualisation

In [8]:
# Функція для вибору випадкових зображень
def get_random_image_path(dataset_path, n_samples=5):
    classes = sorted(os.listdir(dataset_path))
    sample_paths = []
    for _ in range(n_samples):
        random_class = random.choice(classes)
        class_path = os.path.join(dataset_path, random_class)
        if os.path.isdir(class_path):
            random_image = random.choice(os.listdir(class_path))
            image_path = os.path.join(class_path, random_image)
            sample_paths.append(image_path)
    return sample_paths

# Обробка та відображення зображень з кістяками рук
def visualize_hand_landmarks(image_paths):
    for image_path in image_paths:
        # Завантаження зображення
        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Обробка зображення для виявлення рук
        result = hands.process(image_rgb)

        # Малювання кістяка руки на зображенні, якщо знайдено руку
        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image_rgb, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Відображення зображення з кістяком руки
        plt.imshow(image_rgb)
        plt.axis('off')
        plt.show()

# Вибір випадкових зображень для візуалізації
sample_image_paths = get_random_image_path(IMAGE_DATASET_PATH, 3)
visualize_hand_landmarks(sample_image_paths)

# Закриття ресурсу MediaPipe
hands.close()


/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


NameError: name 'mp_drawing' is not defined

# Ploting

In [9]:
def plot_learning_curves(history):
  # Візуалізація кривих точності
  plt.figure(figsize=(12, 6))
  plt.subplot(1, 2, 1)
  plt.plot(history.history['accuracy'], label='Training Accuracy')
  plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
  plt.title('Training and Validation Accuracy')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend()

  # Візуалізація кривих втрат
  plt.subplot(1, 2, 2)
  plt.plot(history.history['loss'], label='Training Loss')
  plt.plot(history.history['val_loss'], label='Validation Loss')
  plt.title('Training and Validation Loss')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()

  plt.show()

# Data preprocessing

In [10]:

# Ініціалізація LabelEncoder
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])

# Розділення даних та міток
X = df.drop('label', axis=1).values
y = df['label'].values

# Розділення на тренувальний та тестовий набори
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'Training samples: {X_train.shape[0]}')
print(f'Testing samples: {X_test.shape[0]}')


Training samples: 1296
Testing samples: 325


In [11]:
# Ініціалізація StandardScaler
scaler = StandardScaler()

# Навчання scaler на тренувальних даних
X_train = scaler.fit_transform(X_train)

# Застосування scaler до тестових даних
X_test = scaler.transform(X_test)

# Перетворення міток у формат one-hot encoding
y_train = to_categorical(y_train, num_classes=36)
y_test = to_categorical(y_test, num_classes=36)

# Розширення вимірів даних для використання в 1D CNN
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)


In [12]:
def get_cnn_model():
  # Створення моделі перед тренуванням
  model = Sequential()

  # Додавання згорткових шарів
  model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Dropout(0.25))

  model.add(Conv1D(128, kernel_size=3, activation='relu'))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Dropout(0.25))

  # Додавання повнозв'язкових шарів
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.5))

  # Вихідний шар для класифікації
  model.add(Dense(36, activation='softmax'))

  return model

model = get_cnn_model()
# Компіляція моделі
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Visualize the CNN model architecture
model.summary()

# Save model visualization to a file
plot_model(model, show_shapes=True, show_layer_names=True)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 61, 64)              │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 30, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 30, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 28, 128)             │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 14, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 14, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1792)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         229,504 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 36)                  │           4,644 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 259,108 (1012.14 KB)

 Trainable params: 259,108 (1012.14 KB)

 Non-trainable params: 0 (0.00 B)

NameError: name 'plot_model' is not defined

In [ ]:
# Після створення моделі - тренування
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=64)

In [ ]:
plot_learning_curves(history)

In [ ]:
# Прогнозуємо на тестовому наборі
y_pred = model.predict(X_test)

# Перетворення one-hot encoded міток у вектори класів
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Create confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)

# Adjusting the figure size for better visibility
plt.figure(figsize=(12, 12))  # Increase the size of the figure

# Create a ConfusionMatrixDisplay
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(y_true))
disp.plot(cmap=plt.cm.Blues)  # 'd' ensures integer display

plt.title('Confusion Matrix', fontsize=16)
plt.show()


In [ ]:
accuracy = accuracy_score(y_true, y_pred_classes)
print(f'Accuracy: {accuracy:.4f}')